In [9]:
import pandas as pd 
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout, BatchNormalization, LeakyReLU
from keras.regularizers import l1_l2
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv("braves_23_sim.csv")
pd.set_option('display.max_columns', None)
df

,game_date,game_pk,player_name,batter,pitcher,stand,p_throws,pit_handR,bat_handR,home_team,away_team,outs_when_up,inning,inning_topbot,launch_speed,launch_angle,hit_direction,is_gb,is_ld,is_fb,is_pu,hangtime,distance,landing_x,landing_y,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,a3,a4,a5,a6,a7,a8,a9,d3,d4,d5,d6,d7,d8,d9,ad3,ad4,ad5,ad6,responsible_fielder,responsible_fielder_x,responsible_fielder_y,responsible_fielder_angle,responsible_fielder_depth,responsible_fielder_coords_diff,responsible_fielder_angle_diff,fielder_landing_x,fielder_landing_y,fielder_landing_angle_radians,is_out
0,2023-10-12,748561,"Johnson, Pierce",592206,572955,R,R,1,1,PHI,ATL,1,8,Bot,97.3,6.0,-39.252699,1,0,0,0,1.218904,130.088910,-82.312694,100.736019,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,71.303273,45.495741,5.777129,24.781611,5,-76.896164,116.285126,-33.475571,139.410368,16.465525,5.777129,-5.416530,-15.549107,-1.906000,1
1,2023-10-09,748563,"Yates, Kirby",592206,489446,R,R,1,1,ATL,PHI,1,5,Top,76.5,26.0,24.728910,0,0,1,0,3.768386,270.677120,113.231124,245.855279,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,7.321664,-18.485868,-58.204480,-39.199998,9,136.387080,260.838400,27.604150,294.343518,27.580649,2.875241,-23.155956,-14.983121,-2.567292,0
2,2023-10-07,748562,"Strider, Spencer",592206,675911,R,R,1,1,ATL,PHI,0,7,Top,94.3,-7.0,-35.186827,1,0,0,0,0.244988,34.460390,-19.857607,28.163697,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,67.237401,41.429869,1.711256,20.715738,5,-76.896164,116.285126,-33.475571,139.410368,104.970393,1.711256,57.038557,-88.121429,-0.996341,1
3,2023-10-07,748562,"Strider, Spencer",592206,675911,R,R,1,1,ATL,PHI,0,5,Top,56.4,-79.0,-51.232253,1,0,0,0,0.079630,1.994988,-1.555473,1.249192,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,83.282827,57.475295,17.756683,36.761165,5,-76.896164,116.285126,-33.475571,139.410368,137.511766,17.756683,75.340691,-115.035934,-0.990962,1
4,2023-10-07,748562,"Strider, Spencer",592206,675911,R,R,1,1,ATL,PHI,2,2,Top,100.6,40.0,22.252696,0,0,1,0,6.102435,348.506770,131.976785,322.550927,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,9.797878,-16.009654,-55.728266,-36.723784,9,136.387080,260.838400,27.604150,294.343518,61.869917,5.351455,-4.410295,61.712527,1.642140,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,2023-04-04,718711,"Dodd, Dylan",660766,689266,R,L,0,1,STL,ATL,2,5,Bot,73.5,22.0,-37.416483,0,1,0,0,3.006265,227.956000,-138.507054,181.051736,62.656757,103.627914,12.065786,154.73065,-77.402350,115.580630,-38.797337,147.21658,-149.08849,256.86870,1.8

In [12]:
outfield_df = df[(df['is_fb'] == 1) | (df['is_ld'] == 1) | (df['is_pu'] == 1)]
outfield_df

,game_date,game_pk,player_name,batter,pitcher,stand,p_throws,pit_handR,bat_handR,home_team,away_team,outs_when_up,inning,inning_topbot,launch_speed,launch_angle,hit_direction,is_gb,is_ld,is_fb,is_pu,hangtime,distance,landing_x,landing_y,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,a3,a4,a5,a6,a7,a8,a9,d3,d4,d5,d6,d7,d8,d9,ad3,ad4,ad5,ad6,responsible_fielder,responsible_fielder_x,responsible_fielder_y,responsible_fielder_angle,responsible_fielder_depth,responsible_fielder_coords_diff,responsible_fielder_angle_diff,fielder_landing_x,fielder_landing_y,fielder_landing_angle_radians,is_out
1,2023-10-09,748563,"Yates, Kirby",592206,489446,R,R,1,1,ATL,PHI,1,5,Top,76.5,26.0,24.728910,0,0,1,0,3.768386,270.67712,113.231124,245.855279,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,7.321664,-18.485868,-58.204480,-39.199998,9,136.38708,260.83840,27.604150,294.343518,27.580649,2.875241,-23.155956,-14.983121,-2.567292,0
4,2023-10-07,748562,"Strider, Spencer",592206,675911,R,R,1,1,ATL,PHI,2,2,Top,100.6,40.0,22.252696,0,0,1,0,6.102435,348.50677,131.976785,322.550927,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,9.797878,-16.009654,-55.728266,-36.723784,9,136.38708,260.83840,27.604150,294.343518,61.869917,5.351455,-4.410295,61.712527,1.642140,1
5,2023-09-20,716509,"Iglesias, Raisel",592206,628452,R,R,1,1,ATL,PHI,0,9,Top,83.3,39.0,-37.546884,0,0,1,0,5.166323,293.50443,-178.864653,232.706432,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,69.597458,43.789926,4.071314,23.075796,7,-139.90384,265.81088,-27.759111,300.380606,51.125819,9.787774,-38.960813,-33.104448,-2.437281,1
6,2023-09-13,716603,"Strider, Spencer",592206,675911,R,R,1,1,PHI,ATL,1,7,Bot,93.5,16.0,34.501869,0,1,0,0,2.968773,266.79330,151.120563,219.866420,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,-2.451295,-28.258827,-67.977439,-48.972957,9,136.38708,260.83840,27.604150,294.343518,43.540540,-6.897718,14.733483,-40.971980,-1.225596,1
7,2023-09-12,716618,"Jiménez, Joe",592206,641729,R,R,1,1,PHI,ATL,1,8,Bot,100.1,16.0,-36.366188,0,1,0,0,2.590281,292.64902,-173.524417,235.653822,64.876760,103.620880,16.846207,153.99442,-76.896164,116.285126,-38.075928,147.53577,-139.90384,265.81088,8.199211,338.15524,136.38708,260.83840,32.050574,6.243042,-33.475571,-14.471088,-27.759111,1.388972,27.604150,122.254983,154.913124,139.410368,152.369878,300.380606,338.254628,294.343518,68.416762,42.609230,2.890617,21.895100,7,-139.90384,265.81088,-27.759111,300.380606,45.164049,8.607077,-33.620577,-30.157058,-2.410447,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,2023-04-05,718697,"McHugh, Collin",666185,543521,L,R,1,0,STL,ATL,2,9,Bot,91.9,15.0,-43.241546,0,1,0,0,2.531397,241.29124,-165.302724,175.773929,73.666890,102.362870,36.986360,148.00433,-67.815155,127.028984,-13.824618,154.48872,-131.84592,264.93948,-6.3674

In [13]:

one_hot_encoder = OneHotEncoder(categories=[['7', '8', '9']], sparse_output=False, handle_unknown='ignore')
responsible_fielder_encoded = one_hot_encoder.fit_transform(outfield_df[['responsible_fielder']])
encoded_df = pd.DataFrame(responsible_fielder_encoded, columns=['is_7', 'is_8', 'is_9'], index=outfield_df.index)
outfield_df = pd.concat([outfield_df, encoded_df], axis=1)
num_features = ['fielder_landing_y', 'fielder_landing_x', 'hangtime']
features = num_features + ['is_7', 'is_8', 'is_9'] 
saved_standard_scaler = StandardScaler().fit(outfield_df[num_features].values)
outfield_df[num_features] = saved_standard_scaler.transform(outfield_df[num_features].values)
outfield_df = outfield_df.drop(['responsible_fielder'], axis=1)
target = 'is_out'

# Separate features and target
X_simplified = outfield_df[features]
y = outfield_df[target]

X_train_simplified, X_test_simplified, y_train, y_test = train_test_split(X_simplified, y, test_size=0.2, random_state=42)


# Define the model
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model
history = model.fit(X_train_simplified, y_train, epochs=110, batch_size=44, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_simplified, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Plotting the training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Epoch 1/110


In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Generate predicted probabilities
y_pred_proba = model.predict(X_test_simplified).flatten()

# Create calibration plot
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)

plt.figure(figsize=(10, 6))
plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='Calibration curve')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly calibrated')
plt.xlabel('Predicted probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration Plot')
plt.legend()
plt.grid()
plt.show()


In [ ]:
#Filtering outfield df for only those plays that resulted in outs 
plot_df = outfield_df[outfield_df["is_out"] == 1]
mean_x = plot_df['responsible_fielder_x'].mean()
mean_y = plot_df['responsible_fielder_y'].mean()

In [ ]:
#Finding Average Actual Coordinates for Outfielders 
left_fielder = outfield_df[outfield_df["is_7"] == 1]
average_x_7 = left_fielder["responsible_fielder_x"].mean()
average_y_7 = left_fielder["responsible_fielder_y"].mean()
cf = outfield_df[outfield_df["is_8"] == 1]
average_x_cf = cf["responsible_fielder_x"].mean()
average_y_cf = cf["responsible_fielder_depth"].mean()
rf = outfield_df[outfield_df["is_9"] == 1]
average_x_rf = rf["responsible_fielder_x"].mean()
average_y_rf = rf["responsible_fielder_depth"].mean()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


# Create StandardScaler instance
# scaler = StandardScaler()

# Generate a grid of landing positions
ht = 3
fielder_x = 0
fielder_y = 0
#fielder = 8  # CFer

x, y = np.meshgrid(np.linspace(fielder_x - 120, fielder_x + 120, 100), np.linspace(fielder_y - 120, fielder_y + 120, 100))
x = x.flatten()
y = y.flatten()
hit_direction_angle = np.arctan2(x, y)

# Create new DataFrame with updated fielder positions
new_df = pd.DataFrame({
    'is_8': np.ones(x.shape[0]),
    'is_9': np.zeros(x.shape[0]),
    'is_7': np.zeros(x.shape[0]),
    'hit_direction': hit_direction_angle,
    'hangtime': np.ones(x.shape[0]) * ht,
    'fielder_landing_x': x,
    'fielder_landing_y': y,
   
})

# Scale numeric features
new_df[num_features] = saved_standard_scaler.transform(new_df[num_features].values)

# Combine all features (one-hot encoded categorical + scaled numeric)
X_all_features = new_df[features]

# Predict the out probabilities
new_df['outprob'] = model.predict(X_all_features).flatten()

# Reshape data for plotting
outprob = new_df['outprob'].values.reshape((100, 100))
x = x.reshape((100, 100))
y = y.reshape((100, 100))

# Plot the out probability heatmap
plt.figure(figsize=(10, 8))
plt.pcolormesh(x, y, outprob, cmap='coolwarm')
plt.colorbar(label='Out Probability')
plt.xlabel('Landing X')
plt.ylabel('Landing Y')
plt.plot([fielder_x], [fielder_y], 'ko', label='Fielder Position')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


# Create StandardScaler instance
# scaler = StandardScaler()

# Generate a grid of landing positions
ht = 4
fielder_x = 0
fielder_y = 0
#fielder = 8  # CFer

x, y = np.meshgrid(np.linspace(fielder_x - 120, fielder_x + 120, 100), np.linspace(fielder_y - 120, fielder_y + 120, 100))
x = x.flatten()
y = y.flatten()
hit_direction_angle = np.arctan2(x, y)

# Create new DataFrame with updated fielder positions
new_df = pd.DataFrame({
    'is_7': np.ones(x.shape[0]),
    'is_9': np.zeros(x.shape[0]),
    'is_8': np.zeros(x.shape[0]),
    'hit_direction': hit_direction_angle,
    'hangtime': np.ones(x.shape[0]) * ht,
    'fielder_landing_x': x,
    'fielder_landing_y': y,
   
})

# Scale numeric features
new_df[num_features] = saved_standard_scaler.transform(new_df[num_features].values)

# Combine all features (one-hot encoded categorical + scaled numeric)
X_all_features = new_df[features]

# Predict the out probabilities
new_df['outprob'] = model.predict(X_all_features).flatten()

# Reshape data for plotting
outprob = new_df['outprob'].values.reshape((100, 100))
x = x.reshape((100, 100))
y = y.reshape((100, 100))

# Plot the out probability heatmap
plt.figure(figsize=(10, 8))
plt.pcolormesh(x, y, outprob, cmap='coolwarm')
plt.colorbar(label='Out Probability')
plt.xlabel('Landing X')
plt.ylabel('Landing Y')
plt.plot([fielder_x], [fielder_y], 'ko', label='Fielder Position')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


# Create StandardScaler instance
# scaler = StandardScaler()

# Generate a grid of landing positions
ht = 4
fielder_x = 0
fielder_y = 0
#fielder = 8  # CFer

x, y = np.meshgrid(np.linspace(fielder_x - 120, fielder_x + 120, 100), np.linspace(fielder_y - 120, fielder_y + 120, 100))
x = x.flatten()
y = y.flatten()
hit_direction_angle = np.arctan2(x, y)

# Create new DataFrame with updated fielder positions
new_df = pd.DataFrame({
    'is_9': np.ones(x.shape[0]),
    'is_7': np.zeros(x.shape[0]),
    'is_8': np.zeros(x.shape[0]),
    'hit_direction': hit_direction_angle,
    'hangtime': np.ones(x.shape[0]) * ht,
    'fielder_landing_x': x,
    'fielder_landing_y': y,
   
})

# Scale numeric features
new_df[num_features] = saved_standard_scaler.transform(new_df[num_features].values)

# Combine all features (one-hot encoded categorical + scaled numeric)
X_all_features = new_df[features]

# Predict the out probabilities
new_df['outprob'] = model.predict(X_all_features).flatten()

# Reshape data for plotting
outprob = new_df['outprob'].values.reshape((100, 100))
x = x.reshape((100, 100))
y = y.reshape((100, 100))

# Plot the out probability heatmap
plt.figure(figsize=(10, 8))
plt.pcolormesh(x, y, outprob, cmap='coolwarm')
plt.colorbar(label='Out Probability')
plt.xlabel('Landing X')
plt.ylabel('Landing Y')
plt.plot([fielder_x], [fielder_y], 'ko', label='Fielder Position')
plt.legend()
plt.show()

In [ ]:
#Hexbin Plot 
plt.figure(figsize=(10, 8))
plt.hexbin(plot_df['landing_x'], plot_df['landing_y'], gridsize=30, cmap='coolwarm', mincnt=1)
plt.scatter(average_x_7, average_y_7, color='black', marker='o', s=100, label='Average Fielder Position')
plt.scatter(average_x_cf, average_y_cf, color='black', marker='o', s=100, label='Average Fielder Position')
plt.scatter(average_x_rf, average_y_rf, color='black', marker='o', s=100, label='Average Fielder Position')
plt.colorbar(label='Count of Outs')
plt.xlabel('Relative X Coordinate')
plt.ylabel('Relative Y Coordinate')
plt.title('Hexbin Plot of Outfield Outs by Relative Coordinates')
plt.show()